In [1]:
import pandas as pd
import numpy as np
import re
import scipy as sp
import sklearn
import sys
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
#from gensim.models import ldamodel
#import gensim.corpora;
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation, PCA
from sklearn import datasets
from sklearn.preprocessing import normalize
import pickle

In [2]:
import pandas as pd
import os
data=pd.read_csv("20190928-reviews.csv")
data.head()

,asin,name,rating,date,verified,title,body,helpfulVotes
0,B0000SX2UC,Janet,3,"October 11, 2005",False,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,1.0
1,B0000SX2UC,Luke Wyatt,1,"January 7, 2004",False,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,17.0
2,B0000SX2UC,Brooke,5,"December 30, 2003",False,Love This Phone,"This is a great, reliable phone. I also purcha...",5.0
3,B0000SX2UC,amy m. teague,3,"March 18, 2004",False,"Love the Phone, BUT...!","I love the phone and all, because I really did...",1.0
4,B0000SX2UC,tristazbimmer,4,"August 28, 2005",False,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...,1.0


In [3]:
data = data.drop(columns=['asin', 'name', 'date','verified','helpfulVotes'], axis=1)
data.head()

,rating,title,body
0,3,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...
1,1,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...
2,5,Love This Phone,"This is a great, reliable phone. I also purcha..."
3,3,"Love the Phone, BUT...!","I love the phone and all, because I really did..."
4,4,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...


In [4]:
data = data.dropna(axis=0, subset=['body'])

In [5]:
def top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

## LSA

In [9]:
tf_idf = TfidfVectorizer(stop_words='english', max_features=1000)
data_tf_idf = tf_idf.fit_transform(data.body)
nmf = NMF(n_components=5, random_state=1,
          alpha=.1, l1_ratio=.5)
nmf.fit(data_tf_idf)
nmf_data=nmf.transform(data_tf_idf)

In [10]:
top_words(nmf, tf_idf.get_feature_names(), 10)

Topic #0: phone battery screen use work just nice camera life best
Topic #1: good product price far quality condition really phone battery value
Topic #2: great phone price works product condition fast value deal far
Topic #3: love phone absolutely new thank note far amazing awesome fast
Topic #4: works new like brand perfect looks condition came excellent just



## PLSA

In [20]:
tf_idf = TfidfVectorizer(stop_words='english', max_features=1000)
data_tf_idf = tf_idf.fit_transform(data.body)
nmf = NMF(n_components=5, random_state=1, beta_loss='kullback-leibler',
          solver='mu', alpha=.1, l1_ratio=.5)
nmf_data=nmf.fit(data_tf_idf)

In [21]:
top_words(nmf, tf_idf.get_feature_names(), 10)

Topic #0: phone screen use work battery just time camera phones samsung
Topic #1: good battery far quality life really price bad pretty value
Topic #2: great works product price value condition far shipping deal shape
Topic #3: love thank great absolutely price far thanks color learning value
Topic #4: new works like perfect excellent came looks condition brand excelente



## LDA

In [22]:
count = CountVectorizer(stop_words='english', max_features=1000)
data_count = count.fit_transform(data.body)
lda = LatentDirichletAllocation(n_components=5, max_iter=5, 
                                learning_method='online', random_state=1)
lda_data=lda.fit_transform(data_count)

In [23]:
top_words(lda, count.get_feature_names(), 10)

Topic #0: phone new like came love charger brand perfect refurbished box
Topic #1: android product google issue device el software version samsung nokia
Topic #2: phone screen camera battery like use quality phones just life
Topic #3: phone work sim card unlocked did verizon just use mobile
Topic #4: great phone good works price far battery fast happy condition

